In [1]:
import os
import warnings
from dotenv import load_dotenv

#If you get duplicate error run---

#os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
#warnings.filterwarnings("ignore")
# ----


env_path = "../OllamaSetup/.env"  
load_dotenv(env_path)

True

In [2]:
from langchain_ollama import OllamaEmbeddings
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

In [3]:
embeddings = OllamaEmbeddings(model= 'nomic-embed-text:latest',base_url='http://localhost:11434')

In [7]:
db_name ="/Users/ragnirupam/Desktop/GenAIProjects/genaienv/VectorStoreAndRetrievals/health_supplements"
#db_name= "health_supplements"
vector_store= FAISS.load_local(db_name,embeddings, allow_dangerous_deserialization=True)

In [8]:
question = "how to gain muscle mass"
docs= vector_store.search(question,k=5, search_type="similarity")
docs

[Document(id='e143abd7-93de-4cae-a226-ecaaae84e469', metadata={'producer': 'iLovePDF', 'creator': '', 'creationdate': '', 'source': 'rag-dataset/gym supplements/2. High Prevalence of Supplement Intake.pdf', 'file_path': 'rag-dataset/gym supplements/2. High Prevalence of Supplement Intake.pdf', 'total_pages': 11, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2024-10-21T11:39:04+00:00', 'trapped': '', 'modDate': 'D:20241021113904Z', 'creationDate': '', 'page': 8}, page_content='and strength gain among men. We detected more prevalent protein and creatine supplementation\namong younger compared to older ﬁtness center users, whereas the opposite was found for vitamin\nsupplementation. Other authors made similar observations [23] and it might reﬂect the diﬀerent\ntraining goals among age groups, i.e., more focus on strength and muscles among the younger and\nmore focus on health among the older age groups.\nComparable to other studies [4], we dete

In [9]:
retriever =vector_store.as_retriever(search_type ='similarity',search_kwargs={'k':3})

retriever.invoke(question)

[Document(id='e143abd7-93de-4cae-a226-ecaaae84e469', metadata={'producer': 'iLovePDF', 'creator': '', 'creationdate': '', 'source': 'rag-dataset/gym supplements/2. High Prevalence of Supplement Intake.pdf', 'file_path': 'rag-dataset/gym supplements/2. High Prevalence of Supplement Intake.pdf', 'total_pages': 11, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2024-10-21T11:39:04+00:00', 'trapped': '', 'modDate': 'D:20241021113904Z', 'creationDate': '', 'page': 8}, page_content='and strength gain among men. We detected more prevalent protein and creatine supplementation\namong younger compared to older ﬁtness center users, whereas the opposite was found for vitamin\nsupplementation. Other authors made similar observations [23] and it might reﬂect the diﬀerent\ntraining goals among age groups, i.e., more focus on strength and muscles among the younger and\nmore focus on health among the older age groups.\nComparable to other studies [4], we dete

In [10]:
question =" How to loose weight"
retriever.invoke(question)

[Document(id='85c771b9-02c8-4d6d-bc7f-5e79a2386f40', metadata={'producer': 'iLovePDF', 'creator': '', 'creationdate': '', 'source': 'rag-dataset/health supplements/1. dietary supplements - for whom.pdf', 'file_path': 'rag-dataset/health supplements/1. dietary supplements - for whom.pdf', 'total_pages': 17, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2024-10-21T11:37:54+00:00', 'trapped': '', 'modDate': 'D:20241021113754Z', 'creationDate': '', 'page': 12}, page_content='Int. J. Environ. Res. Public Health 2021, 18, 8897\n13 of 24\n3.3. Dietary Supplements and Weight Loss\nThe number of people with excessive weight continues to rise, and ﬁghting obesity\nhas become one of the greatest challenges of contemporary medicine. A person wishing to\nlose weight needs to undertake several difﬁcult life-changes and practice them consistently\n(diet, physical activity, addiction-free). Meanwhile dietary supplements are presented as\na compelling altern

In [13]:
retriever =vector_store.as_retriever(search_type ='similarity_score_threshold',
                                     search_kwargs={'k':3 , 'score_threshold' :0.1})

retriever.invoke(question)

[Document(id='85c771b9-02c8-4d6d-bc7f-5e79a2386f40', metadata={'producer': 'iLovePDF', 'creator': '', 'creationdate': '', 'source': 'rag-dataset/health supplements/1. dietary supplements - for whom.pdf', 'file_path': 'rag-dataset/health supplements/1. dietary supplements - for whom.pdf', 'total_pages': 17, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2024-10-21T11:37:54+00:00', 'trapped': '', 'modDate': 'D:20241021113754Z', 'creationDate': '', 'page': 12}, page_content='Int. J. Environ. Res. Public Health 2021, 18, 8897\n13 of 24\n3.3. Dietary Supplements and Weight Loss\nThe number of people with excessive weight continues to rise, and ﬁghting obesity\nhas become one of the greatest challenges of contemporary medicine. A person wishing to\nlose weight needs to undertake several difﬁcult life-changes and practice them consistently\n(diet, physical activity, addiction-free). Meanwhile dietary supplements are presented as\na compelling altern

In [23]:
retriever =vector_store.as_retriever(search_type ='mmr',
                                     search_kwargs={'k':3 , 'fetch_k' :3 ,'lambda_mult' :1})

docs= retriever.invoke(question)

## RAg with LLAMA on Ollama

In [15]:
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain import hub

In [17]:
prompt = hub.pull("rlm/rag-prompt")

In [18]:
#Alternative 


prompt = """ You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:  
         """

In [19]:
prompt= ChatPromptTemplate.from_template(prompt)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template=" You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:  \n         "), additional_kwargs={})])

In [21]:
llm= ChatOllama(model='llama3.2:3b',base_url='http://localhost:11434')
llm.invoke("Hi")

AIMessage(content='How can I assist you today?', additional_kwargs={}, response_metadata={'model': 'llama3.2:3b', 'created_at': '2025-04-01T06:42:50.736818Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3733364250, 'load_duration': 735003250, 'prompt_eval_count': 26, 'prompt_eval_duration': 2810410583, 'eval_count': 8, 'eval_duration': 187066833, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-049c952e-fe60-4140-889a-38114ad0d836-0', usage_metadata={'input_tokens': 26, 'output_tokens': 8, 'total_tokens': 34})

In [24]:
docs

[Document(id='85c771b9-02c8-4d6d-bc7f-5e79a2386f40', metadata={'producer': 'iLovePDF', 'creator': '', 'creationdate': '', 'source': 'rag-dataset/health supplements/1. dietary supplements - for whom.pdf', 'file_path': 'rag-dataset/health supplements/1. dietary supplements - for whom.pdf', 'total_pages': 17, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2024-10-21T11:37:54+00:00', 'trapped': '', 'modDate': 'D:20241021113754Z', 'creationDate': '', 'page': 12}, page_content='Int. J. Environ. Res. Public Health 2021, 18, 8897\n13 of 24\n3.3. Dietary Supplements and Weight Loss\nThe number of people with excessive weight continues to rise, and ﬁghting obesity\nhas become one of the greatest challenges of contemporary medicine. A person wishing to\nlose weight needs to undertake several difﬁcult life-changes and practice them consistently\n(diet, physical activity, addiction-free). Meanwhile dietary supplements are presented as\na compelling altern

In [26]:
def format_docs(docs):
    return '\n\n'.join([doc.page_content for doc in docs])

context= format_docs(docs)
#print(context)

## RAG Chain

In [34]:
rag_chain= (
    {"context":retriever|format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [35]:
question ="how to loose weight"

In [37]:
response= rag_chain.invoke(question)
print(response)

I don't know a definitive way to lose weight as suggested by the context, which highlights the lack of evidence supporting the effectiveness of dietary supplements for weight loss. However, the article suggests that a combination of diet, physical activity, and addiction-free lifestyle is recommended. It's also advised to avoid purchasing products from unauthorized buyers and not increase the dose or use multiple products simultaneously.


In [38]:
question ="what is balanced deit"
response= rag_chain.invoke(question)
print(response)

I don't know the specific answer to your question about "balanced diet" from the provided context. However, I can tell you that the context does mention phytochemicals and food supplements, which are related to nutrition and health benefits. If you could provide more information or clarify your question, I'll do my best to help.


In [43]:
#Alternative 


prompt = """ You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know. 
Answer in bullet points.
Make sure your answer is relevant to the question and it is answered from the context only
Question: {question} 
Context: {context} 
Answer:  
         """

prompt= ChatPromptTemplate.from_template(prompt)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template=" You are an assistant for question-answering tasks. \nUse the following pieces of retrieved context to answer the question.\nIf you don't know the answer, just say that you don't know. \nAnswer in bullet points.\nMake sure your answer is relevant to the question and it is answered from the context only\nQuestion: {question} \nContext: {context} \nAnswer:  \n         "), additional_kwargs={})])

In [44]:
question ="how to loose weight?"
response= rag_chain.invoke(question)
print(response)

I don't know a guaranteed way to lose weight, but I can provide general advice. To lose weight, it's recommended to adopt a balanced diet, combine it with regular physical activity, and maintain an addiction-free lifestyle. Consult reputable sources or healthcare professionals for personalized guidance on safe and effective weight loss strategies.
